<a href="https://colab.research.google.com/github/sebastian-dv/CSE-151A-Project/blob/main/Model3_SVM_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSE 151A Group Project
Names
Pranav Prabu
Sebastian Diaz
Jou-Chih Chang
Juan Yin
Irving Zhao
Xianzhe Guo
Tiankuo Li

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [50]:
df = pd.read_csv('https://archive.ics.uci.edu/static/public/880/data.csv')
df = df[['age','sex','death','dzgroup','scoma','race','sps','aps','diabetes','dementia','meanbp','wblc','hrt','resp','temp','pafi','alb','bili','crea','sod','ph']]

In [51]:
# one hot encoding race
ohe = pd.get_dummies(df[['race']])
df = df.drop(['race'], axis = 1)
df = pd.concat([df, ohe], axis = 1)

# dropping nan values
df = df.dropna(axis = 0, how = 'any')

# encoding sex
df['sex'].replace('female', 0, inplace=True)
df['sex'].replace('male', 1, inplace=True)

# one hot encoding dzgroup
ohe = pd.get_dummies(df[['dzgroup']])
df = df.drop(['dzgroup'], axis = 1)

X = df.copy()
y = ohe.copy()
df = pd.concat([df, ohe], axis = 1)

### MinMaxScaler

In [52]:
# scaling data
# scaler = MinMaxScaler()
# X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
# X_scaled = scaler.fit_transform(X)
# X.head()


### StandardScaler

In [53]:

non_num_X = X.iloc[:, 19:]
num_X = X.drop(X.columns[19:], axis=1)
num_X = num_X.drop(columns=['sex'])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
StandardScaler()

X_scaled = scaler.fit_transform(num_X)
X_scaled = np.concatenate([X_scaled, non_num_X.values], axis=1)


In [54]:
non_num_X

,race_asian,race_black,race_hispanic,race_other,race_white
0,0,0,0,1,0
6,0,0,0,0,1
17,0,0,0,0,1
22,0,0,0,0,1
25,0,0,0,0,1
...,...,...,...,...,...
9091,0,0,0,0,1
9099,0,0,0,0,1
9100,0,0,0,0,1
9103,0,0,0,0,1


In [55]:
X_scaled

array([[ 0.05296858, -1.39942757, -0.5261451 , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.02305731,  0.71457789, -0.5261451 , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.10373612,  0.71457789,  0.47997099, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.25422663, -1.39942757, -0.5261451 , ...,  0.        ,
         0.        ,  1.        ],
       [-0.93551828,  0.71457789, -0.5261451 , ...,  0.        ,
         0.        ,  1.        ],
       [ 1.21998078,  0.71457789, -0.5261451 , ...,  0.        ,
         0.        ,  1.        ]])

In [56]:
# split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=21)

In [57]:
# 1. Build a linear kernel SVM
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# C is the SVM regularization parameter
C = 0.1  # able to change
# Create an Instance of SVM and Fit out the data.
# Data is not scaled so as to be able to plot the support vectors
svm = SVC(kernel ='linear')
svm.fit(X_train,y_train.idxmax(axis=1).values)
y_true = y_test.idxmax(axis=1).values
y_pred = svm.predict(X_test)
print(classification_report(y_true, y_pred))
# Evaluate the accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

                           precision    recall  f1-score   support

dzgroup_ARF/MOSF w/Sepsis       0.55      0.83      0.66       333
              dzgroup_CHF       0.38      0.38      0.38        96
             dzgroup_COPD       0.37      0.30      0.33        83
        dzgroup_Cirrhosis       0.00      0.00      0.00        49
     dzgroup_Colon Cancer       0.50      0.06      0.10        18
             dzgroup_Coma       0.73      0.53      0.62        77
      dzgroup_Lung Cancer       0.26      0.29      0.27        42
     dzgroup_MOSF w/Malig       0.00      0.00      0.00        70

                 accuracy                           0.51       768
                macro avg       0.35      0.30      0.29       768
             weighted avg       0.43      0.51      0.45       768

Accuracy: 0.5091145833333334


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Try to find optimiaze parameters


In [59]:
# find the beter parameters
# test for best model
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1], 'kernel': ['linear', 'rbf', 'poly']}
# Create an SVM classifier
svm_classifier = SVC()
# Create the GridSearchCV object
grid_search = GridSearchCV(svm_classifier, param_grid, cv=3, scoring='accuracy')
# Fit the model with different hyperparameter combinations
grid_search.fit(X_train,y_train.idxmax(axis=1).values)
# Get the best hyperparameters
best_params = grid_search.best_params_
# Get the best model
best_model = grid_search.best_estimator_
# Evaluate the best model on the test set
accuracy = best_model.score(X_test, y_test.idxmax(axis=1).values)
print(f"Best Hyperparameters: {best_params}")
print(f"Accuracy on Test Set: {accuracy}")

Best Hyperparameters: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}
Accuracy on Test Set: 0.53515625


## OverSamplying with scaled data

### SMOTE


In [67]:
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Apply SMOTE to oversample the minority class
smote = SMOTE(random_state=21)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train.idxmax(axis=1).values)
# After oversampling
unique_classes_resampled, class_counts_resampled = np.unique(y_train_resampled, return_counts=True)
print(f"Class distribution after oversampling: {dict(zip(unique_classes_resampled, class_counts_resampled))}")
for class_label, count in zip(unique_classes_resampled, class_counts_resampled):
    print(f"Frequency of Class {class_label}: {count} instances")

Class distribution after oversampling: {'dzgroup_ARF/MOSF w/Sepsis': 1392, 'dzgroup_CHF': 1392, 'dzgroup_COPD': 1392, 'dzgroup_Cirrhosis': 1392, 'dzgroup_Colon Cancer': 1392, 'dzgroup_Coma': 1392, 'dzgroup_Lung Cancer': 1392, 'dzgroup_MOSF w/Malig': 1392}
Frequency of Class dzgroup_ARF/MOSF w/Sepsis: 1392 instances
Frequency of Class dzgroup_CHF: 1392 instances
Frequency of Class dzgroup_COPD: 1392 instances
Frequency of Class dzgroup_Cirrhosis: 1392 instances
Frequency of Class dzgroup_Colon Cancer: 1392 instances
Frequency of Class dzgroup_Coma: 1392 instances
Frequency of Class dzgroup_Lung Cancer: 1392 instances
Frequency of Class dzgroup_MOSF w/Malig: 1392 instances


### RandomOverSampler

In [77]:
import numpy as np
from imblearn.over_sampling import RandomOverSampler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

rs = RandomOverSampler(random_state=11)
X_train_resampled, y_train_resampled = rs.fit_resample(X_train, y_train.idxmax(axis=1).values)
# After oversampling
unique_classes_resampled, class_counts_resampled = np.unique(y_train_resampled, return_counts=True)
print(f"Class distribution after oversampling: {dict(zip(unique_classes_resampled, class_counts_resampled))}")
for class_label, count in zip(unique_classes_resampled, class_counts_resampled):
    print(f"Frequency of Class {class_label}: {count} instances")


Class distribution after oversampling: {'dzgroup_ARF/MOSF w/Sepsis': 1392, 'dzgroup_CHF': 1392, 'dzgroup_COPD': 1392, 'dzgroup_Cirrhosis': 1392, 'dzgroup_Colon Cancer': 1392, 'dzgroup_Coma': 1392, 'dzgroup_Lung Cancer': 1392, 'dzgroup_MOSF w/Malig': 1392}
Frequency of Class dzgroup_ARF/MOSF w/Sepsis: 1392 instances
Frequency of Class dzgroup_CHF: 1392 instances
Frequency of Class dzgroup_COPD: 1392 instances
Frequency of Class dzgroup_Cirrhosis: 1392 instances
Frequency of Class dzgroup_Colon Cancer: 1392 instances
Frequency of Class dzgroup_Coma: 1392 instances
Frequency of Class dzgroup_Lung Cancer: 1392 instances
Frequency of Class dzgroup_MOSF w/Malig: 1392 instances


### Evaluation

In [78]:
svm_classifier = SVC(kernel='rbf', C = 10, gamma = 0.1)
svm_classifier.fit(X_train_resampled,y_train_resampled)
y_true = y_test.idxmax(axis=1).values
y_pred = svm_classifier.predict(X_test)
print(classification_report(y_true, y_pred))
# Evaluate the accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy}")

                           precision    recall  f1-score   support

dzgroup_ARF/MOSF w/Sepsis       0.61      0.75      0.67       333
              dzgroup_CHF       0.48      0.43      0.45        96
             dzgroup_COPD       0.32      0.29      0.30        83
        dzgroup_Cirrhosis       0.50      0.37      0.42        49
     dzgroup_Colon Cancer       0.06      0.06      0.06        18
             dzgroup_Coma       0.56      0.40      0.47        77
      dzgroup_Lung Cancer       0.20      0.17      0.18        42
     dzgroup_MOSF w/Malig       0.36      0.27      0.31        70

                 accuracy                           0.51       768
                macro avg       0.39      0.34      0.36       768
             weighted avg       0.49      0.51      0.49       768

Accuracy: 0.5091145833333334
